# Estimating School Location

Integration with [larch](https://larch.newman.me) for model estimation. See [estimation tools review](https://github.com/ActivitySim/activitysim/wiki/Estimation-Tools-Review) for more information about larch.

# Run the Example

Output an estimation data bundle (EBD), which contains:
  - model settings - school_location_model_settings.yaml
  - coefficients - school_location_coefficients.csv
  - utilities specification - school_location_SPEC.csv
  - land use data - school_location_landuse.csv
  - size terms - school_location_size_terms.csv
  - alternatives values - school_location_alternatives_combined.csv
  - chooser data - school_location_choosers_combined.csv
  - chooses made - school_location_choices.csv

# Read EDB

In [1]:
import larch  # !conda install larch #for estimation
import pandas as pd
import numpy as np
import yaml 
import larch.util.excel
import larch_asim  # utility functions in a local module
import os

In [2]:
from larch import P,X

In [3]:
edb_directory = "estimation_data_bundle/school_location/"

def read_csv(filename, **kwargs):
    return pd.read_csv(os.path.join(edb_directory, filename), **kwargs)

## Read Settings

In [4]:
try:
    from yaml import CLoader as yamlLoader
except ImportError:
    from yaml import yamlLoader

with open(os.path.join(edb_directory, "school_location_model_settings.yaml"), 'r') as stream:
    settings = yaml.load(stream, Loader=yamlLoader)


In [5]:
CHOOSER_SEGMENT_COLUMN_NAME = settings['CHOOSER_SEGMENT_COLUMN_NAME']
SEGMENT_IDS = settings['SEGMENT_IDS']
settings

{'SAMPLE_SIZE': 30,
 'SIMULATE_CHOOSER_COLUMNS': ['TAZ', 'school_segment', 'household_id'],
 'CHOOSER_ORIG_COL_NAME': 'TAZ',
 'ALT_DEST_COL_NAME': 'alt_dest',
 'IN_PERIOD': 14,
 'OUT_PERIOD': 8,
 'DEST_CHOICE_COLUMN_NAME': 'school_taz',
 'SAMPLE_SPEC': 'school_location_sample.csv',
 'SPEC': 'school_location.csv',
 'COEFFICIENTS': 'school_location_coeffs.csv',
 'LOGSUM_SETTINGS': 'tour_mode_choice.yaml',
 'LOGSUM_PREPROCESSOR': 'nontour_preprocessor',
 'LOGSUM_TOUR_PURPOSE': {'university': 'univ',
  'highschool': 'school',
  'gradeschool': 'school'},
 'annotate_persons': {'SPEC': 'annotate_persons_school', 'DF': 'persons'},
 'CHOOSER_TABLE_NAME': 'persons',
 'MODEL_SELECTOR': 'school',
 'CHOOSER_SEGMENT_COLUMN_NAME': 'school_segment',
 'CHOOSER_FILTER_COLUMN_NAME': 'is_student',
 'SEGMENT_IDS': {'university': 3, 'highschool': 2, 'gradeschool': 1},
 'SHADOW_PRICE_TABLE': 'school_shadow_prices',
 'MODELED_SIZE_TABLE': 'school_modeled_size',
 'SAVED_SHADOW_PRICE_TABLE_NAME': 'school_shadow

## Read EDB

### Spec

In [6]:
spec = read_csv("school_location_SPEC.csv")

In [7]:
spec

,Label,Description,Expression,university,highschool,gradeschool
0,local_dist,NaN,_DIST@skims['DIST'],1,1,1
1,util_dist_0_1,"Distance, piecewise linear from 0 to 1 miles","@_DIST.clip(0,1)",coef_univ_dist_0_1,coef_high_dist_0_1,coef_grade_dist_0_1
2,util_dist_1_2,"Distance, piecewise linear from 1 to 2 miles","@(_DIST-1).clip(0,1)",coef_univ_dist_1_2,coef_high_grade_dist_1_2,coef_high_grade_dist_1_2
3,util_dist_2_5,"Distance, piecewise linear from 2 to 5 miles","@(_DIST-2).clip(0,3)",coef_univ_dist_2_5,coef_high_grade_dist_2_5,coef_high_grade_dist_2_5
4,util_dist_5_15,"Distance, piecewise linear from 5 to 15 miles","@(_DIST-5).clip(0,10)",coef_univ_dist_5_15,coef_high_dist_5_15,coef_grade_dist_5_15
5,util_dist_15_up,"Distance, piecewise linear for 15+ miles",@(_DIST-15.0).clip(0),coef_univ_dist_15_up,coef_high_dist_15_up,coef_grade_dist_15_up
6,util_size_variable,Size variable,@(df['size_term'] * df['shadow_price_size_term...,1,1,1
7,util_utility_adjustment,utility adjustment,@df['shadow_price_utility_adjustment'],1,1,1
8,util_no_attractions,No attractions,@df['size_term']==0,-999,-999,-999
9,util_mode_choice_logsum,Mode choice logsum,mode_choice_logsum,coef_mode_logsum,coef_mode_logsum,coef_mode_logsum


In [8]:
# Remove shadow pricing and pre-existing size expression

spec = spec\
.set_index('Label')\
.drop(index=['util_size_variable', 'util_utility_adjustment'])\
.reset_index()

### Size Spec

In [9]:
size_spec = read_csv("school_location_size_terms.csv")

In [10]:
school_size_spec = size_spec \
.query("model_selector == 'school'") \
.drop(columns='model_selector') \
.set_index('segment')
school_size_spec = school_size_spec.loc[:,school_size_spec.max()>0]
school_size_spec

,AGE0519,HSENROLL,COLLFTE,COLLPTE
segment,,,,
university,0.0,0.0,0.592,0.408
gradeschool,1.0,0.0,0.000,0.000
highschool,0.0,1.0,0.000,0.000


### Size Coefficients

In [11]:
size_coef = school_size_spec.stack().reset_index()
size_coef.index = size_coef.iloc[:,0] +"_"+ size_coef.iloc[:,1]
size_coef = size_coef.loc[size_coef.iloc[:,2]>0]
size_coef['constrain'] = 'F'
one_each = size_coef.groupby('segment').first().reset_index()
size_coef.loc[one_each.iloc[:,0] +"_"+ one_each.iloc[:,1], 'constrain'] = 'T'
size_coef = size_coef.iloc[:,2:]
size_coef.columns = ['value','constrain']
size_coef.index.name = 'coefficient_name'
size_coef['value'] = np.log(size_coef['value'])
size_coef

,value,constrain
coefficient_name,,
university_COLLFTE,-0.524249,T
university_COLLPTE,-0.896488,F
gradeschool_AGE0519,0.000000,T
highschool_HSENROLL,0.000000,T


### Coefficients

In [12]:
coefficients = pd.read_csv(
    "estimation_data_bundle/school_location/school_location_coefficients.csv",
    index_col=0
)
coefficients

,value,constrain
coefficient_name,,
coef_univ_dist_0_1,-3.2451,F
coef_univ_dist_1_2,-2.7011,F
coef_univ_dist_2_5,-0.5707,F
coef_univ_dist_5_15,-0.5002,F
coef_univ_dist_15_up,-0.0730,F
coef_high_dist_0_1,-0.9523,F
coef_high_grade_dist_1_2,-0.5700,F
coef_high_grade_dist_2_5,-0.5700,F
coef_high_dist_5_15,-0.1930,F


### Choosers

In [13]:
x_co = read_csv(
    "school_location_choosers_combined.csv",
    index_col='person_id'
)
x_co.head()

,TAZ,school_segment,household_id,model_choice,override_choice
person_id,,,,,
29448,16,3,29448,101,12
30431,31,3,30431,13,9
31152,46,3,31152,12,9
31532,62,3,31532,69,69
31609,63,3,31609,9,69


In [14]:
x_co.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1041 entries, 29448 to 7540206
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   TAZ              1041 non-null   int64
 1   school_segment   1041 non-null   int64
 2   household_id     1041 non-null   int64
 3   model_choice     1041 non-null   int64
 4   override_choice  1041 non-null   int64
dtypes: int64(5)
memory usage: 48.8 KB


### Alternative Features

In [15]:
x_cv = pd.read_csv(
    "estimation_data_bundle/school_location/school_location_alternatives_combined.csv.gz",
    index_col=(0,1)
)
x_cv.head()

1  \
person_id variable                                                 
29448     TAZ                                                1.0   
          mode_choice_logsum                 0.09343871150392973   
          pick_count                                         1.0   
          prob                                               0.0   
          shadow_price_size_term_adjustment                  1.0   

                                                               2  \
person_id variable                                                 
29448     TAZ                                                2.0   
          mode_choice_logsum                 0.25638125423874897   
          pick_count                                         1.0   
          prob                                               0.0   
          shadow_price_size_term_adjustment                  1.0   

                                                               3  \
person_id variable                                                 
29448     TAZ                                                3.0   
          mode_choice_logsum                 0.07610352691757298   
          pick_count                                         1.0   
          prob                                               0.0   
          shadow_price_size_term_adjustment                  1.0   

                                                               4  \
person_id variable                                                 
29448     TAZ                                                4.0   
          mode_choice_logsum                 0.20451243745701267   
          pick_count                                         1.0   
          prob                                               0.0   
          shadow_price_size_term_adjustment                  1.0   

                                                                 5  \
person_id variable                                                   
29448     TAZ                                                  5.0   
          mode_choice_logsum                   0.12339758011229311   
          pick_count                                           1.0   
          prob                               0.0018385813330181202   
          shadow_price_size_term_adjustment                    1.0   

                                                                6  \
person_id variable                                                  
29448     TAZ                                                 6.0   
          mode_choice_logsum                 -0.17359392001161128   
          pick_count                                          1.0   
          prob                                                0.0   
          shadow_price_size_term_adjustment                   1.0   

                                                               7  \
person_id variable                                                 
29448     TAZ                                                7.0   
          mode_choice_logsum                 -0.0765343018299863   
          pick_count                                         1.0   
          prob                                               0.0   
          shadow_price_size_term_adjustment                  1.0   

                                                                8  \
person_id variable                                                  
29448     TAZ                                                 8.0   
          mode_choice_logsum                 -0.29563326714849975   
          pick_count                                          1.0   
          prob                                                0.0   
          shadow_price_size_term_adjustment                   1.0   

                                                                9  \
person_id variable                                                  
29448     TAZ                                                 9.0   
          mo

### Land Use

In [16]:
landuse = read_csv("school_location_landuse.csv", index_col='TAZ')
landuse.head()

,DISTRICT,SD,county_id,TOTHH,TOTPOP,TOTACRE,RESACRE,CIACRE,TOTEMP,AGE0519,...,OPRKCST,area_type,HSENROLL,COLLFTE,COLLPTE,TOPOLOGY,TERMINAL,household_density,employment_density,density_index
TAZ,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,46,82,20.3,1.0,15.00000,27318,7,...,932.83514,0,0.0,0.00000,0.0,3,5.89564,2.875000,1707.375000,2.870167
2,1,1,1,134,240,31.1,1.0,24.79297,42078,19,...,885.61682,0,0.0,0.00000,0.0,1,5.84871,5.195214,1631.374751,5.178722
3,1,1,1,267,476,14.7,1.0,2.31799,2445,38,...,716.27252,0,0.0,0.00000,0.0,1,5.53231,80.470405,736.891913,72.547987
4,1,1,1,151,253,19.3,1.0,18.00000,22434,20,...,314.00000,0,0.0,0.00000,0.0,2,5.64330,7.947368,1180.736842,7.894233
5,1,1,1,611,1069,52.7,1.0,15.00000,15662,86,...,314.01431,0,0.0,72.14684,0.0,1,5.52555,38.187500,978.875000,36.753679


## Prep Data

In [17]:
x_ca = larch_asim.cv_to_ca(x_cv)

In [18]:
x_ca_1 = pd.merge(x_ca, landuse, on='TAZ', how='left')
x_ca_1.index = x_ca.index

In [19]:
x_ca_1, x_co = larch_asim.prevent_overlapping_column_names(x_ca_1, x_co)

In [20]:
d = larch.DataFrames(
    co=x_co,
    ca=x_ca_1,
    av=True,
)

## Prep Model

In [21]:
m = larch.Model()

In [22]:
m.utility_ca = larch_asim.linear_utility_from_spec(
    spec, x_col='Label', 
    p_col=SEGMENT_IDS, 
    ignore_x=('local_dist',), 
    segment_id=CHOOSER_SEGMENT_COLUMN_NAME,
)
print(m.utility_ca)

  P.coef_univ_dist_0_1 * X('util_dist_0_1*(school_segment==3)')
+ P.coef_univ_dist_1_2 * X('util_dist_1_2*(school_segment==3)')
+ P.coef_univ_dist_2_5 * X('util_dist_2_5*(school_segment==3)')
+ P.coef_univ_dist_5_15 * X('util_dist_5_15*(school_segment==3)')
+ P.coef_univ_dist_15_up * X('util_dist_15_up*(school_segment==3)')
+ P('-999') * X('util_no_attractions*(school_segment==3)')
+ P.coef_mode_logsum * X('util_mode_choice_logsum*(school_segment==3)')
+ P('1') * X('util_sample_of_corrections_factor*(school_segment==3)')
+ P.coef_high_dist_0_1 * X('util_dist_0_1*(school_segment==2)')
+ P.coef_high_grade_dist_1_2 * X('util_dist_1_2*(school_segment==2)')
+ P.coef_high_grade_dist_2_5 * X('util_dist_2_5*(school_segment==2)')
+ P.coef_high_dist_5_15 * X('util_dist_5_15*(school_segment==2)')
+ P.coef_high_dist_15_up * X('util_dist_15_up*(school_segment==2)')
+ P('-999') * X('util_no_attractions*(school_segment==2)')
+ P.coef_mode_logsum * X('util_mode_choice_logsum*(school_segment==2)')
+ P(

In [23]:
m.quantity_ca = sum(
    P(f"{i}_{q}") * X(q) * X(f"{settings['CHOOSER_SEGMENT_COLUMN_NAME']}=={settings['SEGMENT_IDS'][i]}")
    for i in school_size_spec.index
    for q in school_size_spec.columns
    if school_size_spec.loc[i,q]!=0
)

In [24]:
larch_asim.explicit_value_parameters_from_spec(spec, p_col=SEGMENT_IDS, model=m)

In [25]:
m.pf

,value,initvalue,nullvalue,minimum,maximum,holdfast,note
-999,-999.0,0.0,0.0,-inf,inf,1,
1,1.0,0.0,0.0,-inf,inf,1,
coef_grade_dist_0_1,0.0,0.0,0.0,-inf,inf,0,
coef_grade_dist_15_up,0.0,0.0,0.0,-inf,inf,0,
coef_grade_dist_5_15,0.0,0.0,0.0,-inf,inf,0,
coef_high_dist_0_1,0.0,0.0,0.0,-inf,inf,0,
coef_high_dist_15_up,0.0,0.0,0.0,-inf,inf,0,
coef_high_dist_5_15,0.0,0.0,0.0,-inf,inf,0,
coef_high_grade_dist_1_2,0.0,0.0,0.0,-inf,inf,0,
coef_high_grade_dist_2_5,0.0,0.0,0.0,-inf,inf,0,


In [26]:
larch_asim.apply_coefficients(coefficients, m)
larch_asim.apply_coefficients(size_coef, m, minimum=-6, maximum=6)

In [27]:
m.pf  # Spot check, confirm coefficients set correctly. 

,value,initvalue,nullvalue,minimum,maximum,holdfast,note
-999,-999.000000,-999.0,-999.0,-999.0,-999.0,1,
1,1.000000,1.0,1.0,1.0,1.0,1,
coef_grade_dist_0_1,-1.641900,0.0,0.0,NaN,NaN,0,
coef_grade_dist_15_up,-0.046000,0.0,0.0,NaN,NaN,0,
coef_grade_dist_5_15,-0.203100,0.0,0.0,NaN,NaN,0,
coef_high_dist_0_1,-0.952300,0.0,0.0,NaN,NaN,0,
coef_high_dist_15_up,-0.188200,0.0,0.0,NaN,NaN,0,
coef_high_dist_5_15,-0.193000,0.0,0.0,NaN,NaN,0,
coef_high_grade_dist_1_2,-0.570000,0.0,0.0,NaN,NaN,0,
coef_high_grade_dist_2_5,-0.570000,0.0,0.0,NaN,NaN,0,


In [28]:
m.dataservice = d

In [29]:
m.choice_co_code = 'override_choice'

## Re-Estimate

In [30]:
m.estimate()

req_data does not request avail_ca or avail_co but it is set and being provided


,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
-999,-999.000000,-999.0,-999.0,-999.0,-999.0,1,,-999.000000
1,1.000000,1.0,1.0,1.0,1.0,1,,1.000000
coef_grade_dist_0_1,-4.312078,0.0,0.0,NaN,NaN,0,,-4.312078
coef_grade_dist_15_up,-0.046000,0.0,0.0,NaN,NaN,0,,-0.046000
coef_grade_dist_5_15,-0.362956,0.0,0.0,NaN,NaN,0,,-0.362956
coef_high_dist_0_1,-1.661998,0.0,0.0,NaN,NaN,0,,-1.661998
coef_high_dist_15_up,-0.188200,0.0,0.0,NaN,NaN,0,,-0.188200
coef_high_dist_5_15,-0.458801,0.0,0.0,NaN,NaN,0,,-0.458801
coef_high_grade_dist_1_2,-1.220211,0.0,0.0,NaN,NaN,0,,-1.220211
coef_high_grade_dist_2_5,-1.359651,0.0,0.0,NaN,NaN,0,,-1.359651


c:\programdata\anaconda3\envs\asimtest-develop-tnc-updates-2\lib\site-packages\larch\linalg\__init__.py:18: UserWarning: minimum eig 9.98186850774182e-16 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")


┣                x: -999                       -999.000000
┃                   1                             1.000000
┃                   coef_grade_dist_0_1          -4.312078
┃                   coef_grade_dist_15_up        -0.046000
┃                   coef_grade_dist_5_15         -0.362956
┃                   coef_high_dist_0_1           -1.661998
┃                   coef_high_dist_15_up         -0.188200
┃                   coef_high_dist_5_15          -0.458801
┃                   coef_high_grade_dist_1_2     -1.220211
┃                   coef_high_grade_dist_2_5     -1.359651
┃                   coef_mode_logsum              0.175216
┃                   coef_univ_dist_0_1           -5.474144
┃                   coef_univ_dist_15_up         -0.073000
┃                   coef_univ_dist_1_2           -5.355847
┃                   coef_univ_dist_2_5           -1.195146
┃                   coef_univ_dist_5_15          -1.421225
┃                   gradeschool_AGE0519           0.000000
┃                   highschool_HSENROLL           0.000000
┃                   university_COLLFTE           -0.524249
┃                   university_COLLPTE            2.080543
┃                   dtype: float64
┣          loglike: -3183.502239747373
┣        d_loglike: -999                        0.000000
┃                   1                           0.000000
┃                   coef_grade_dist_0_1        -0.000327
┃                   coef_grade_dist_15_up       0.000000
┃                   coef_grade_dist_5_15        0.000443
┃                   coef_high_dist_0_1          0.000332
┃                   coef_high_dist_15_up        0.000000
┃                   coef_high_dist_5_15        -0.001173
┃                   coef_high_grade_dist_1_2    0.001978
┃                   coef_high_grade_dist_2_5    0.009383
┃                   coef_mode_logsum            0.008761
┃                   coef_univ_dist_0_1         -0.000016
┃                   coef_univ_dist_15_up        0.000000
┃                   coef_univ_dist_1_2         -0.000456
┃                   coef_univ_dist_2_5         -0.000741
┃                   coef_univ_dist_5_15         0.000167
┃                   gradeschool_AGE0519         0.000000
┃                   highschool_HSENROLL         0.000000
┃                   university_COLLFTE          0.000000
┃                   university_COLLPTE         -0.002138
┃                   dtype: float64
┣              nit: 20
┣             nfev: 43
┣             njev: 20
┣           status: 0
┣          message: 'Optimization terminated successfully.'
┣          success: True
┣     elapsed_time: datetime.timedelta(seconds=2, microseconds=299160)
┣           method: 'slsqp'
┣          n_cases: 1041
┣ iteration_number: 20
┣          logloss: 3.058119346539263

In [31]:
m.possible_overspecification

## Write Out Results

In [32]:
# Write re-estimated value back into the coefficients file.
est_names = [j for j in coefficients.index if j in m.pf.index]
coefficients.loc[est_names, 'value'] = m.pf.loc[est_names, 'value']

In [33]:
# Write out replacement coefficients file and model summaries
os.makedirs(os.path.join(edb_directory,'estimated'), exist_ok=True)

In [34]:
coefficients.reset_index().to_csv(
    os.path.join(
        edb_directory, 
        'estimated',
        "school_location_coefficients_revised.csv",
    ),
    index=False,
)

In [35]:
m.to_xlsx(
    os.path.join(
        edb_directory, 
        'estimated',
        "school_location_model_estimation.xlsx",
    )
)

In [36]:
# Write size coefficients into size_spec
for c in school_size_spec.columns:
    for i in school_size_spec.index:
        param_name = f"{i}_{c}"
        j = (size_spec['segment'] == i) & (size_spec['model_selector'] == 'school')
        try:
            size_spec.loc[j,c] = np.exp(m.get_value(param_name))
        except KeyError:
            pass


In [37]:
# Rescale each row to total 1, not mathematically needed
# but to maintain a consistent approach from existing ASim

size_spec.iloc[:,2:] = (size_spec.iloc[:,2:].div(size_spec.iloc[:,2:].sum(1), axis=0))

In [38]:
size_spec.to_csv(
    os.path.join(edb_directory,'estimated',"school_location_size_terms.csv"), 
    index=False,
)